In [43]:
import cv2
import numpy as np

In [44]:
p = []
output_width = 800

def get_desired_height(frame):
    return int(output_width / (frame.shape[1] / frame.shape[0]))

def point_selection(event, x, y, flags, param):
    frame = param[0]  

    if event == cv2.EVENT_LBUTTONDOWN:
        p.append((x, y))
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  
        cv2.imshow('frame', frame)  

def get_all_corners(frame):
    cv2.namedWindow('frame')
    old_frame_shape = frame.shape[1]
    frame = cv2.resize(frame, (output_width, get_desired_height(frame)))
    cv2.setMouseCallback('frame', point_selection, [frame])
    
    while True:
        cv2.imshow('frame', frame)
        if len(p) == 4 or (cv2.waitKey(30) & 0xFF == 27):
            break

    cv2.destroyAllWindows()
    return np.float32(np.array(p) / output_width * old_frame_shape)

def track_corners(prev_frame, frame, corners):
    prev = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    new_points, status, _ = cv2.calcOpticalFlowPyrLK(prev, curr, corners, None)

    if status is not None and new_points is not None:
        indices = np.where(status.flatten() == 1)[0]
        return new_points[indices]
    else:
        return np.empty(0)

def replace_by_homography(frame, corners, headshot):
    h, w, _ = headshot.shape
    headshot_corners = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    
    matrix, status = cv2.findHomography(headshot_corners, corners)

    if status is None:
        return frame
    else:
        warp = cv2.warpPerspective(headshot, matrix, (frame.shape[1], frame.shape[0]))
        mask = np.zeros_like(frame, dtype=np.uint8)
        cv2.fillConvexPoly(mask, np.int32(corners), (255, 255, 255))

        background = cv2.subtract(frame, mask)  
        replaced_image = np.where(mask != 0, warp, 0)

        return cv2.add(background, replaced_image)


In [ ]:
def main():
    cap = cv2.VideoCapture('KandinskyBook.mp4')
    headshot = cv2.imread('mbappe.jpg')

    ret, frame = cap.read()
    if not ret:
        return
    # frame = cv2.resize(frame, (output_width, get_desired_height(frame)))
    corners = get_all_corners(frame)

    prev_frame = frame
    out = cv2.VideoWriter('q5_out2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (frame.shape[1], frame.shape[0]))

    while cap.isOpened():
        # grab a frame
        ret, frame = cap.read() 
        if not ret:
            return
        # frame = cv2.resize(frame, (output_width, get_desired_height(frame)))
        
        if ret == True: 
            new_corners = track_corners(prev_frame, frame, corners)
            if len(new_corners) < 4:
                continue

            corners = new_corners
            img = replace_by_homography(frame, corners, headshot)

            out.write(img)

            img = cv2.imshow('frame', img)
            if cv2.waitKey(30) & 0xFF == 27: # wait 1 ms and exit is ESC is pressed
                break

            prev_frame = frame
        else:
            break
    out.release()
    cap.release()
    cv2.destroyAllWindows()
main()
